In [ ]:

def parse_survey_answers_files():
    df = pd.read_csv('./data/survey.csv')
    df.dropna()
    df = df.drop(
        ['W skali od 1 do 10 jak bardzo lubisz słone jedzenie', 'W skali od 1 do 10 jak bardzo lubisz słodkie jedzenie',
         'W skali od 1 do 10 jak bardzo lubisz gorzkie jedzenie', 'W skali od 1 do 10 jak bardzo lubisz mięso',
         'W skali od 1 do 10 jak bardzo lubisz pikantne jedzenie',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie śródziemnomorską',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie azjatycką',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie amerykańską',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie arabską',
         'W skali od 1 do 10 jak bardzo lubisz owoce morza(również ryby)',
         'W skali od 1 do 10 jak bardzo lubisz tłuste jedzenie', 'W skali od 1 do 10 jak bardzo lubisz kuchnie Polską'
         ],
        axis=1)
    columns = df.columns
    df.T.insert(0, "questions", columns)
    return df

parse_survey_answers_files().head()


In [6]:
import pandas as pd
from src.utils import load_and_preprocess_data
from engine.recommender import Recommender
from scipy import spatial

In [7]:
data, email_order, dishes_order, sorted_users, sorted_dishes = load_and_preprocess_data()
recommender = Recommender(
        data['ocena'],
        email_order,
        dishes_order,
    )

FileNotFoundError: [Errno 2] No such file or directory: './src/data/ratings.csv'

In [5]:
recommender.create_and_fit(
        # Fine-tuned values
        model_params=dict(
            factors=20,
            alpha=0.6,
            regularization=0.06,
        ),
)

NameError: name 'recommender' is not defined

In [4]:
suggestions_and_score  = recommender.recommend_products(
                20, 40
            )


# for i in range(len(suggestions_and_score[1])):
#     print(suggestions_and_score[0][i], "   ", suggestions_and_score[1][i])
#     print(sorted_dishes[suggestions_and_score[0][i]])

In [5]:
def compare_taste_with_taste_profile(dish_name_list, user_email, user_profiles_df: pd.DataFrame = None,
                                     user_profiles_path: str = './data/user_profiles.csv',
                                     recipes_df: pd.DataFrame = None, recipes_path: str = './data/recipes.csv'):
    if not user_profiles_df:
        user_profiles_df = pd.read_csv(user_profiles_path)
    user_profile = (user_profiles_df.loc[user_profiles_df['email'] == user_email][
                        ["saltiness", "bitterness", 'spiciness', 'fattiness']].values * 10)[0]
    if not recipes_df:
        recipes_df = pd.read_csv(recipes_path)
    cosine_similarity_list = list()
    for dish_name in dish_name_list:
        dish = \
            recipes_df.loc[recipes_df['title'] == dish_name][
                ["saltiness", "bitterness", 'spiciness', 'fattiness']].values[
                0]
        cosine_similarity_list.append((1 - spatial.distance.cosine(user_profile, dish), dish_name))
    cosine_similarity_list.sort(key=lambda x: x[0], reverse=True)
    return cosine_similarity_list


compare_taste_with_taste_profile([sorted_dishes[i] for i in suggestions_and_score[0].tolist()], sorted_users[20])


[(0.8276961252801738, 'Beef Wellington'),
 (0.8270715804662274, 'Vegan Colcannon Soup'),
 (0.826416979086038, 'X-Country Double Lobster Risotto'),
 (0.8255045512803283, 'Colcannon'),
 (0.8249217329670919, 'Slow Cooker Shanghai-Style Beef Borscht'),
 (0.8248401403603774, 'Austrian Goulash'),
 (0.8215499893426359, 'Spanish Tortilla'),
 (0.8208052386056948, 'Hungarian Beef Goulash'),
 (0.8203281048058185, 'Easy Weeknight Beef Stroganoff'),
 (0.8190835074709936, 'Sheet Pan Butternut Squash with Spicy Italian Sausage'),
 (0.818322138758193, 'Gnocchi and Wilted Spinach Salad'),
 (0.8181758430417793, 'Baked Chicken Caprese'),
 (0.8161270368569296,
  'Spring Risotto with Shrimp, Asparagus and Artichoke Hearts'),
 (0.8138330212896766, 'Gluten-Free Lamb Moussaka'),
 (0.8068162279998052, 'Hungarian Potato Soup'),
 (0.8056793300738222, 'Sauteed Italian Eggplant Side Dish'),
 (0.7958477017765462, 'Pierogi Casserole'),
 (0.7926557231657605, 'Italian Seafood Stew'),
 (0.7691188553638618, 'Beef Stroga

In [ ]:
from data.survey import update_survey_answers_file
update_survey_answers_file()


In [ ]:
from data.survey import convert_to_implicit_dataset
convert_to_implicit_dataset().to_csv('./data/ratings.csv')

In [ ]:
from data.recipes import get_all_recipes
test = get_all_recipes('f1a65dd7832647d18888c171f92ed1b7')